# CUNY MSDS DATA618 - Quantative Finance

## Week 3 : Portfolio 

### Keeno Glanville
---------------------
You are a 37-year-old bond trader in a major investment firm. Your investment portfolio, largely holding high return equities is valued today at $6.2 M. You have just read Jamie Dimon's financial hurricane warning and decided to rebalance your portfolio.

## Library to be used
var 2023.6.2
pip install var

### Steps
* Utilize yfinance to obtain historical data with date as index, and tickers as column. Chose one property (Adj Close)
* Convert the data into a percent change
* Adjust using weight of current portfolio

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import scipy.stats

import yfinance as yf
from var import VaR

d:\MSDS\msds_venv\Lib\site-packages\var\methods.py:22: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(cache=True)
d:\MSDS\msds_venv\Lib\site-packages\var\methods.py:53: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(cache=True)
d:\MSDS\msds_venv\Lib\site-packages\var\methods.py:87: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/kglan/MSDS/main/DATA618/portfolio.csv')
df = df[:-3]

In [3]:
df.head()

,Asset,Ticker,Type,Subtype,Date,Price,Units,Amount
0,Apple Inc.,AAPL,Equity,Technology,12-Sep-23,$176.30,648,"$114,242"
1,Amazon.com Inc.,AMZN,Equity,E-commerce,12-Sep-23,$142.32,"1,000","$142,320"
2,Microsoft Corporation,MSFT,Equity,Technology,12-Sep-23,$331.77,500,"$165,885"
3,Alphabet Inc.,GOOGL,Equity,Technology,12-Sep-23,$135.19,800,"$108,152"
4,META,META,Equity,Technology,12-Sep-23,$324.40,500,"$162,200"


In [4]:
# List of all tickers from your provided data
tickers = ['AAPL', 'AMZN', 'MSFT', 'GOOGL', 'META', 'TSLA', 'NVDA', 'AMD', 'JPM', 'JNJ', 'V', 'PG', 'JCI', 'GE', 'KO', 'VZ', 'BA', 'T', 'PFE', 'NFLX', 'BABA', 'INTC', 'DIS', 'XOM', 'F', 'GS', 'HD', 'IBM', 'SBUX', 'CSCO', 'LQD', 'HYG', 'IEF', 'BND', 'SHY', 'VCSH', 'BKLN', 'TIP', 'SCHP', 'TLT']

# Define the date range for historical data
start_date = '2022-01-01'
end_date = '2023-09-12'

# Create an empty DataFrame to store the historical data
historical_data = pd.DataFrame()

# Loop through the tickers, download historical data, and concatenate it to the DataFrame
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    historical_data[ticker] = data["Adj Close"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
historical_data['Cash'] = 1

In [6]:
# Calculate daily returns from adjusted closing prices
daily_returns = historical_data.pct_change()

# Remove the first row (NaN) since there's no return for the first day
daily_returns = daily_returns[1:]
daily_returns.index = pd.to_datetime(daily_returns.index)

In [7]:
# Remove dollar signs and commas from the 'Amount' column and convert to float
df['Amount'] = df['Amount'].str.replace('[$,]', '', regex=True).astype(float)
total_portfolio_value = df['Amount'].sum()



# Weight of Portfolio
df['Weight'] = df['Amount'] / total_portfolio_value
weights = df['Weight'].values

In [8]:
var = VaR(daily_returns, weights, alpha=[0.05])
var

<VaR - μ: 0.01%, σ: 1.1395%, Portfolio σ: 1.1408%>

In [10]:
# Calculate VaR using various methods
print(var.historic(), var.parametric(), var.monte_carlo(), var.monte_carlo(stressed=True), sep='\n')


            VaR(95.0)  CVaR(95.0)  CDaR(95.0)
2023-09-11  -0.018029   -0.024314   -0.240734
            VaR(95.0)  CVaR(95.0)  CDaR(95.0)
2023-09-11  -0.037458         NaN   -0.240734
            VaR(95.0)  CVaR(95.0)  CDaR(95.0)
2023-09-11  -0.018614   -0.023436   -0.240734
            VaR(95.0)  CVaR(95.0)  CDaR(95.0)
2023-09-11  -0.027276   -0.029759   -0.671511


The values provided in your results are the calculated estimates of these risk metrics for a specific date, which is September 11, 2023. The negative values indicate potential losses in the portfolio 